Сначала загружаем и импортим все необходимое.

In [1]:
!pip install -q -U watermark

ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
markdown 3.3.6 requires importlib-metadata>=4.4; python_version < "3.10", but you have importlib-metadata 2.1.3 which is incompatible.


In [2]:
!pip install transformers

     |████████████████████████████████| 3.8 MB 5.2 MB/s 
     |████████████████████████████████| 67 kB 6.9 MB/s 
     |████████████████████████████████| 596 kB 64.8 MB/s 
     |████████████████████████████████| 6.5 MB 40.6 MB/s 
     |████████████████████████████████| 895 kB 32.3 MB/s 
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13


In [3]:
!pip install datasets

     |████████████████████████████████| 325 kB 5.3 MB/s 
     |████████████████████████████████| 1.1 MB 67.8 MB/s 
     |████████████████████████████████| 134 kB 74.6 MB/s 
     |████████████████████████████████| 212 kB 53.2 MB/s 
     |████████████████████████████████| 127 kB 54.8 MB/s 
     |████████████████████████████████| 271 kB 53.9 MB/s 
     |████████████████████████████████| 94 kB 3.0 MB/s 
     |████████████████████████████████| 144 kB 41.2 MB/s 
  Attempting uninstall: urllib3
    Found existing installation: urllib3 1.24.3
    Uninstalling urllib3-1.24.3:
      Successfully uninstalled urllib3-1.24.3
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
datascience 0.10.6 requires folium==0.2.1, but you have folium 0.8.3 which is incompatible.


In [4]:
%reload_ext watermark
%watermark -v -p numpy,pandas,torch,transformers

Python implementation: CPython
Python version       : 3.7.13
IPython version      : 5.5.0

numpy       : 1.21.5
pandas      : 1.3.5
torch       : 1.10.0+cu111
transformers: 4.17.0



In [5]:
!wget https://drive.google.com/uc?id=1zdmewp7ayS4js4VtrJEHzAheSW-5NBZv 
!gdown --id 1S6qMioqPJjyBLpLVz4gmRTnJHnjitnuV
!gdown --id 1zdmewp7ayS4js4VtrJEHzAheSW-5NBZv

--2022-03-27 18:02:58--  https://drive.google.com/uc?id=1zdmewp7ayS4js4VtrJEHzAheSW-5NBZv
Resolving drive.google.com (drive.google.com)... 173.194.74.102, 173.194.74.113, 173.194.74.100, ...
Connecting to drive.google.com (drive.google.com)|173.194.74.102|:443... connected.
HTTP request sent, awaiting response... 303 See Other
Location: https://doc-0o-64-docs.googleusercontent.com/docs/securesc/ha0ro937gcuc7l7deffksulhg5h7mbp1/586enclg21tqtmpmc7vsj1jqhqqgk6n9/1648404150000/01328360181817234523/*/1zdmewp7ayS4js4VtrJEHzAheSW-5NBZv [following]
--2022-03-27 18:03:01--  https://doc-0o-64-docs.googleusercontent.com/docs/securesc/ha0ro937gcuc7l7deffksulhg5h7mbp1/586enclg21tqtmpmc7vsj1jqhqqgk6n9/1648404150000/01328360181817234523/*/1zdmewp7ayS4js4VtrJEHzAheSW-5NBZv
Resolving doc-0o-64-docs.googleusercontent.com (doc-0o-64-docs.googleusercontent.com)... 108.177.112.132, 2607:f8b0:4001:c12::84
Connecting to doc-0o-64-docs.googleusercontent.com (doc-0o-64-docs.googleusercontent.com)|108.177.112.1

In [68]:
import transformers
from transformers import BertModel, AutoTokenizer, BertTokenizer, PreTrainedTokenizerFast, AdamW, get_linear_schedule_with_warmup, DistilBertModel, BertConfig
import torch.nn.functional as F

import torch
import numpy as np
import pandas as pd
import seaborn as sns
from pylab import rcParams
import matplotlib.pyplot as plt
from matplotlib import rc
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix, classification_report
from collections import defaultdict
from textwrap import wrap
from torch import nn, optim
from torch.nn.utils import clip_grad_norm_
from torch.utils.data import Dataset, DataLoader
from google.colab import data_table
data_table.disable_dataframe_formatter()
from transformers import AutoTokenizer
import datasets
from transformers import BertForSequenceClassification, TrainingArguments, Trainer, DataCollatorWithPadding
from sklearn.metrics import precision_recall_fscore_support, accuracy_score

Задаем кол-во классов, переводим оценки в классы, как в исходной тетрадке.

In [7]:
class_num = 3

In [8]:
df = pd.read_csv("reviews.csv")

In [9]:
def to_sentiment(rating):
  rating = int(rating)
  if rating <= 2:
    return 0
  elif rating == 3:
    return 1
  else: 
    return 2

df['sentiment'] = df.score.apply(to_sentiment)

Определяем предобученную модель, пока что я взяла bert-tiny.

In [10]:
PRE_TRAINED_MODEL_NAME = 'prajjwal1/bert-tiny'

Загружаем ее токенизатор.

In [11]:
tokenizer = BertTokenizer.from_pretrained(PRE_TRAINED_MODEL_NAME)

Downloading:   0%|          | 0.00/226k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/285 [00:00<?, ?B/s]

Трэйн-тест сплит.

In [12]:
df_train, df_test = train_test_split(df, test_size=0.1)
df_val, df_test = train_test_split(df_test, test_size=0.5)

Функция препроцессинга с токенизатором, как в исходной тетрадке. Здесь же определяем паддинг.

In [13]:
def preprocess_function(examples):
     return tokenizer(examples["text"], padding=True)

Переименовываю некоторые колонки так, как того ожидает модель, и загружаю датасеты с помощью HugginFace-овских функций.

In [14]:
df_train['text'] = df_train['content']
df_test['text'] = df_test['content']
df_val['text'] = df_val['content']
df_train['labels'] = df_train['sentiment']
df_test['labels'] = df_test['sentiment']
df_val['labels'] = df_val['sentiment']
dataset_train = datasets.Dataset.from_pandas(df_train)
dataset_val = datasets.Dataset.from_pandas(df_val)
dataset_test = datasets.Dataset.from_pandas(df_test)

Препроцессим.

In [15]:
train_tok = dataset_train.map(preprocess_function, batched=True)
test_tok = dataset_test.map(preprocess_function, batched=True)
val_tok = dataset_val.map(preprocess_function, batched=True)

  0%|          | 0/15 [00:00<?, ?ba/s]

  0%|          | 0/1 [00:00<?, ?ba/s]

  0%|          | 0/1 [00:00<?, ?ba/s]

Чтобы модель можно было обучать через Trainer, нужно, чтоб она выдавала кортеж, первый элемент которого -- лосс, а второй непосредственно предсказания. Я просто буду считать CrossEntropyLoss прямо внутри forward-a.

In [16]:
device = torch.device('cuda:0')
loss_fn = nn.CrossEntropyLoss().to(device)

class SentimentClassifier(nn.Module):

  def __init__(self, n_classes):
    super().__init__()
    self.bert = BertModel.from_pretrained(PRE_TRAINED_MODEL_NAME)
    self.drop = nn.Dropout(p=0.3)
    self.out = nn.Linear(self.bert.config.hidden_size, n_classes)
  
  def forward(self, input_ids, attention_mask, labels):
    sequence_output, pooled_output = self.bert(
      input_ids=input_ids,
      attention_mask=attention_mask,
      return_dict=False)
    output = self.drop(pooled_output)
    return (loss_fn(self.out(output), labels), self.out(output))

Задаем модель, переносим на ГПУ. Задаем data_collator, он тоже нужен для Trainer-a.

In [53]:
model = SentimentClassifier(class_num)
model = model.to(device)
data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

loading configuration file https://huggingface.co/prajjwal1/bert-tiny/resolve/main/config.json from cache at /root/.cache/huggingface/transformers/3cf34679007e9fe5d0acd644dcc1f4b26bec5cbc9612364f6da7262aed4ef7a4.a5a11219cf90aae61ff30e1658ccf2cb4aa84d6b6e947336556f887c9828dc6d
Model config BertConfig {
  "attention_probs_dropout_prob": 0.1,
  "classifier_dropout": null,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 128,
  "initializer_range": 0.02,
  "intermediate_size": 512,
  "layer_norm_eps": 1e-12,
  "max_position_embeddings": 512,
  "model_type": "bert",
  "num_attention_heads": 2,
  "num_hidden_layers": 2,
  "pad_token_id": 0,
  "position_embedding_type": "absolute",
  "transformers_version": "4.17.0",
  "type_vocab_size": 2,
  "use_cache": true,
  "vocab_size": 30522
}

loading weights file https://huggingface.co/prajjwal1/bert-tiny/resolve/main/pytorch_model.bin from cache at /root/.cache/huggingface/transformers/1ee037c9e1a220d5c814779ffe697080d1e6f5b16

Кастомная функция для подсчета прочих метрик из исходной тетрадки, ее тоже дадим Trainer-у.

In [54]:
def compute_metrics(pred):
    labels = pred.label_ids
    preds = pred.predictions.argmax(-1)
    precision, recall, f1, _ = precision_recall_fscore_support(labels, preds, average='weighted')
    acc = accuracy_score(labels, preds)
    return {
        'accuracy': acc,
        'f1': f1,
        'precision': precision,
        'recall': recall
    }

Задаем TrainingArguments, они у меня будут одинаковыми для всех моделей (как и требовалось в задании).

In [55]:
training_args = TrainingArguments(
    output_dir="./results",
    evaluation_strategy="epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=32,
    per_device_eval_batch_size=32,
    num_train_epochs=8,
    weight_decay=0.01,
    logging_steps=1
    )

PyTorch: setting up devices
The default value for the training argument `--report_to` will change in v5 (from all installed integrations to none). In v5, you will need to use `--report_to all` to get the same behavior as now. You should start updating your code and make this info disappear :-).


In [56]:
train_tok.set_format(type='torch', columns=['input_ids', 'attention_mask', 'labels'])
test_tok.set_format(type='torch', columns=['input_ids', 'attention_mask', 'labels'])
val_tok.set_format(type='torch', columns=['input_ids', 'attention_mask', 'labels'])

## 1

Задаем Trainer, обучаемся.

In [57]:
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_tok,
    eval_dataset=test_tok,
    tokenizer=tokenizer,
    data_collator=data_collator,
    compute_metrics=compute_metrics
    )

trainer.train()

The following columns in the training set  don't have a corresponding argument in `SentimentClassifier.forward` and have been ignored: score, at, repliedAt, sentiment, __index_level_0__, token_type_ids, content, sortOrder, userName, appId, text, userImage, replyContent, thumbsUpCount, reviewCreatedVersion. If score, at, repliedAt, sentiment, __index_level_0__, token_type_ids, content, sortOrder, userName, appId, text, userImage, replyContent, thumbsUpCount, reviewCreatedVersion are not expected by `SentimentClassifier.forward`,  you can safely ignore this message.
/usr/local/lib/python3.7/dist-packages/transformers/optimization.py:309: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  FutureWarning,
***** Running training *****
  Num examples = 14171
  Num Epochs = 8
  Instantaneous batch size per device = 32
  Total t

Epoch,Training Loss,Validation Loss,Accuracy,F1,Precision,Recall
1,1.052100,0.974797,0.529188,0.473469,0.479212,0.529188
2,0.857700,0.852952,0.579949,0.541990,0.546207,0.579949
3,0.848600,0.816172,0.595178,0.570154,0.568760,0.595178
4,0.830300,0.801879,0.600254,0.576744,0.573235,0.600254
5,0.983900,0.779453,0.612944,0.601947,0.597441,0.612944
6,0.674500,0.770988,0.626904,0.619825,0.616806,0.626904
7,0.778200,0.771634,0.633249,0.616642,0.614237,0.633249
8,0.615000,0.769446,0.630711,0.615152,0.612350,0.630711


The following columns in the evaluation set  don't have a corresponding argument in `SentimentClassifier.forward` and have been ignored: score, at, repliedAt, sentiment, __index_level_0__, token_type_ids, content, sortOrder, userName, appId, text, userImage, replyContent, thumbsUpCount, reviewCreatedVersion. If score, at, repliedAt, sentiment, __index_level_0__, token_type_ids, content, sortOrder, userName, appId, text, userImage, replyContent, thumbsUpCount, reviewCreatedVersion are not expected by `SentimentClassifier.forward`,  you can safely ignore this message.
***** Running Evaluation *****
  Num examples = 788
  Batch size = 32
Saving model checkpoint to ./results/checkpoint-500
Trainer.model is not a `PreTrainedModel`, only saving its state dict.
tokenizer config file saved in ./results/checkpoint-500/tokenizer_config.json
Special tokens file saved in ./results/checkpoint-500/special_tokens_map.json
The following columns in the evaluation set  don't have a corresponding argumen

TrainOutput(global_step=3544, training_loss=0.8546191104445985, metrics={'train_runtime': 187.9154, 'train_samples_per_second': 603.293, 'train_steps_per_second': 18.86, 'total_flos': 0.0, 'train_loss': 0.8546191104445985, 'epoch': 8.0})

In [58]:
metrics = trainer.evaluate(eval_dataset=val_tok, metric_key_prefix="test")
print(f'''metrics:\n
accuracy: {metrics['test_accuracy']},\n
f1: {metrics['test_f1']},\n
precision: {metrics['test_precision']},\n
recall: {metrics['test_recall']},\n''')

The following columns in the evaluation set  don't have a corresponding argument in `SentimentClassifier.forward` and have been ignored: score, at, repliedAt, sentiment, __index_level_0__, token_type_ids, content, sortOrder, userName, appId, text, userImage, replyContent, thumbsUpCount, reviewCreatedVersion. If score, at, repliedAt, sentiment, __index_level_0__, token_type_ids, content, sortOrder, userName, appId, text, userImage, replyContent, thumbsUpCount, reviewCreatedVersion are not expected by `SentimentClassifier.forward`,  you can safely ignore this message.
***** Running Evaluation *****
  Num examples = 787
  Batch size = 32


metrics:

accuracy: 0.6581956797966964,

f1: 0.64760094356727,

precision: 0.6509942766505324,

recall: 0.6581956797966964,



## 2

Практически такая же модель, но здесь мы еще достаем эмбеддинг CLS-токена и конкатенируем его с выходом пулинг-слоя, прежде чем подать это все на вход дропауту и затем линейному слою.

In [59]:
class SentimentClassifier_2(nn.Module):

  def __init__(self, n_classes):
    super().__init__()
    self.bert = BertModel.from_pretrained(PRE_TRAINED_MODEL_NAME)
    self.drop = nn.Dropout(p=0.3)
    self.out = nn.Linear(self.bert.config.hidden_size*2, n_classes)
  
  def forward(self, input_ids, attention_mask, labels):
    sequence_output, pooled_output = self.bert(
      input_ids=input_ids,
      attention_mask=attention_mask,
      return_dict=False)
    cls_emb = sequence_output[:,0,:]
    output = self.drop(torch.cat((cls_emb, pooled_output), 1))
    return (loss_fn(self.out(output), labels), self.out(output))

In [60]:
model_2 = SentimentClassifier_2(class_num)
model_2 = model_2.to(device)

loading configuration file https://huggingface.co/prajjwal1/bert-tiny/resolve/main/config.json from cache at /root/.cache/huggingface/transformers/3cf34679007e9fe5d0acd644dcc1f4b26bec5cbc9612364f6da7262aed4ef7a4.a5a11219cf90aae61ff30e1658ccf2cb4aa84d6b6e947336556f887c9828dc6d
Model config BertConfig {
  "attention_probs_dropout_prob": 0.1,
  "classifier_dropout": null,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 128,
  "initializer_range": 0.02,
  "intermediate_size": 512,
  "layer_norm_eps": 1e-12,
  "max_position_embeddings": 512,
  "model_type": "bert",
  "num_attention_heads": 2,
  "num_hidden_layers": 2,
  "pad_token_id": 0,
  "position_embedding_type": "absolute",
  "transformers_version": "4.17.0",
  "type_vocab_size": 2,
  "use_cache": true,
  "vocab_size": 30522
}

loading weights file https://huggingface.co/prajjwal1/bert-tiny/resolve/main/pytorch_model.bin from cache at /root/.cache/huggingface/transformers/1ee037c9e1a220d5c814779ffe697080d1e6f5b16

In [61]:
trainer_2 = Trainer(
    model=model_2,
    args=training_args,
    train_dataset=train_tok,
    eval_dataset=test_tok,
    tokenizer=tokenizer,
    data_collator=data_collator,
    compute_metrics=compute_metrics
    )

trainer_2.train()

The following columns in the training set  don't have a corresponding argument in `SentimentClassifier_2.forward` and have been ignored: score, at, repliedAt, sentiment, __index_level_0__, token_type_ids, content, sortOrder, userName, appId, text, userImage, replyContent, thumbsUpCount, reviewCreatedVersion. If score, at, repliedAt, sentiment, __index_level_0__, token_type_ids, content, sortOrder, userName, appId, text, userImage, replyContent, thumbsUpCount, reviewCreatedVersion are not expected by `SentimentClassifier_2.forward`,  you can safely ignore this message.
/usr/local/lib/python3.7/dist-packages/transformers/optimization.py:309: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  FutureWarning,
***** Running training *****
  Num examples = 14171
  Num Epochs = 8
  Instantaneous batch size per device = 32
  Tot

Epoch,Training Loss,Validation Loss,Accuracy,F1,Precision,Recall
1,0.995200,0.966552,0.531726,0.504113,0.510354,0.531726
2,0.806700,0.858658,0.588832,0.562656,0.564160,0.588832
3,0.911200,0.813757,0.625635,0.609794,0.608051,0.625635
4,0.749300,0.799603,0.621827,0.609037,0.607357,0.621827
5,0.960000,0.780505,0.640863,0.636191,0.633835,0.640863
6,0.661200,0.768480,0.643401,0.641325,0.639674,0.643401
7,0.898500,0.766230,0.644670,0.637134,0.633995,0.644670
8,0.645900,0.764410,0.647208,0.641270,0.638427,0.647208


The following columns in the evaluation set  don't have a corresponding argument in `SentimentClassifier_2.forward` and have been ignored: score, at, repliedAt, sentiment, __index_level_0__, token_type_ids, content, sortOrder, userName, appId, text, userImage, replyContent, thumbsUpCount, reviewCreatedVersion. If score, at, repliedAt, sentiment, __index_level_0__, token_type_ids, content, sortOrder, userName, appId, text, userImage, replyContent, thumbsUpCount, reviewCreatedVersion are not expected by `SentimentClassifier_2.forward`,  you can safely ignore this message.
***** Running Evaluation *****
  Num examples = 788
  Batch size = 32
Saving model checkpoint to ./results/checkpoint-500
Trainer.model is not a `PreTrainedModel`, only saving its state dict.
tokenizer config file saved in ./results/checkpoint-500/tokenizer_config.json
Special tokens file saved in ./results/checkpoint-500/special_tokens_map.json
The following columns in the evaluation set  don't have a corresponding arg

TrainOutput(global_step=3544, training_loss=0.8633529583736413, metrics={'train_runtime': 187.4338, 'train_samples_per_second': 604.843, 'train_steps_per_second': 18.908, 'total_flos': 0.0, 'train_loss': 0.8633529583736413, 'epoch': 8.0})

In [62]:
metrics = trainer_2.evaluate(eval_dataset=val_tok, metric_key_prefix="test")
print(f'''metrics:\n
accuracy: {metrics['test_accuracy']},\n
f1: {metrics['test_f1']},\n
precision: {metrics['test_precision']},\n
recall: {metrics['test_recall']},\n''')

The following columns in the evaluation set  don't have a corresponding argument in `SentimentClassifier_2.forward` and have been ignored: score, at, repliedAt, sentiment, __index_level_0__, token_type_ids, content, sortOrder, userName, appId, text, userImage, replyContent, thumbsUpCount, reviewCreatedVersion. If score, at, repliedAt, sentiment, __index_level_0__, token_type_ids, content, sortOrder, userName, appId, text, userImage, replyContent, thumbsUpCount, reviewCreatedVersion are not expected by `SentimentClassifier_2.forward`,  you can safely ignore this message.
***** Running Evaluation *****
  Num examples = 787
  Batch size = 32


metrics:

accuracy: 0.6747141041931385,

f1: 0.6689899021559856,

precision: 0.6676611928511442,

recall: 0.6747141041931385,



## 3

Теперь третья модель, здесь просто загружаем ту же самую предобученную модель в класс, уже заточенный на классификацию текстов (BertForSequenceClassification).

In [63]:
model_3 = BertForSequenceClassification.from_pretrained(PRE_TRAINED_MODEL_NAME, num_labels=class_num)

loading configuration file https://huggingface.co/prajjwal1/bert-tiny/resolve/main/config.json from cache at /root/.cache/huggingface/transformers/3cf34679007e9fe5d0acd644dcc1f4b26bec5cbc9612364f6da7262aed4ef7a4.a5a11219cf90aae61ff30e1658ccf2cb4aa84d6b6e947336556f887c9828dc6d
Model config BertConfig {
  "attention_probs_dropout_prob": 0.1,
  "classifier_dropout": null,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 128,
  "id2label": {
    "0": "LABEL_0",
    "1": "LABEL_1",
    "2": "LABEL_2"
  },
  "initializer_range": 0.02,
  "intermediate_size": 512,
  "label2id": {
    "LABEL_0": 0,
    "LABEL_1": 1,
    "LABEL_2": 2
  },
  "layer_norm_eps": 1e-12,
  "max_position_embeddings": 512,
  "model_type": "bert",
  "num_attention_heads": 2,
  "num_hidden_layers": 2,
  "pad_token_id": 0,
  "position_embedding_type": "absolute",
  "transformers_version": "4.17.0",
  "type_vocab_size": 2,
  "use_cache": true,
  "vocab_size": 30522
}

loading weights file https://huggi

In [64]:
trainer_3 = Trainer(
    model=model_3,
    args=training_args,
    train_dataset=train_tok,
    eval_dataset=test_tok,
    tokenizer=tokenizer,
    data_collator=data_collator,
    compute_metrics=compute_metrics
    )

trainer_3.train()

The following columns in the training set  don't have a corresponding argument in `BertForSequenceClassification.forward` and have been ignored: score, at, repliedAt, sentiment, __index_level_0__, content, sortOrder, userName, appId, text, userImage, replyContent, thumbsUpCount, reviewCreatedVersion. If score, at, repliedAt, sentiment, __index_level_0__, content, sortOrder, userName, appId, text, userImage, replyContent, thumbsUpCount, reviewCreatedVersion are not expected by `BertForSequenceClassification.forward`,  you can safely ignore this message.
/usr/local/lib/python3.7/dist-packages/transformers/optimization.py:309: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  FutureWarning,
***** Running training *****
  Num examples = 14171
  Num Epochs = 8
  Instantaneous batch size per device = 32
  Total train batch s

Epoch,Training Loss,Validation Loss,Accuracy,F1,Precision,Recall
1,1.035200,0.982419,0.532995,0.518520,0.515151,0.532995
2,0.890300,0.874018,0.590102,0.559997,0.568469,0.590102
3,0.913900,0.833219,0.615482,0.596272,0.598759,0.615482
4,0.775000,0.813263,0.612944,0.599345,0.596869,0.612944
5,0.934700,0.798130,0.624365,0.615667,0.613283,0.624365
6,0.663100,0.787758,0.625635,0.617319,0.615234,0.625635
7,0.778700,0.782962,0.629442,0.618201,0.616117,0.629442
8,0.630000,0.781825,0.628173,0.617696,0.616028,0.628173


The following columns in the evaluation set  don't have a corresponding argument in `BertForSequenceClassification.forward` and have been ignored: score, at, repliedAt, sentiment, __index_level_0__, content, sortOrder, userName, appId, text, userImage, replyContent, thumbsUpCount, reviewCreatedVersion. If score, at, repliedAt, sentiment, __index_level_0__, content, sortOrder, userName, appId, text, userImage, replyContent, thumbsUpCount, reviewCreatedVersion are not expected by `BertForSequenceClassification.forward`,  you can safely ignore this message.
***** Running Evaluation *****
  Num examples = 788
  Batch size = 32
Saving model checkpoint to ./results/checkpoint-500
Configuration saved in ./results/checkpoint-500/config.json
Model weights saved in ./results/checkpoint-500/pytorch_model.bin
tokenizer config file saved in ./results/checkpoint-500/tokenizer_config.json
Special tokens file saved in ./results/checkpoint-500/special_tokens_map.json
The following columns in the evalua

TrainOutput(global_step=3544, training_loss=0.8482522110772187, metrics={'train_runtime': 193.4733, 'train_samples_per_second': 585.962, 'train_steps_per_second': 18.318, 'total_flos': 136078549051302.0, 'train_loss': 0.8482522110772187, 'epoch': 8.0})

In [65]:
metrics = trainer_3.evaluate(eval_dataset=val_tok, metric_key_prefix="test")
print(f'''metrics:\n
accuracy: {metrics['test_accuracy']},\n
f1: {metrics['test_f1']},\n
precision: {metrics['test_precision']},\n
recall: {metrics['test_recall']},\n''')

The following columns in the evaluation set  don't have a corresponding argument in `BertForSequenceClassification.forward` and have been ignored: score, at, repliedAt, sentiment, __index_level_0__, content, sortOrder, userName, appId, text, userImage, replyContent, thumbsUpCount, reviewCreatedVersion. If score, at, repliedAt, sentiment, __index_level_0__, content, sortOrder, userName, appId, text, userImage, replyContent, thumbsUpCount, reviewCreatedVersion are not expected by `BertForSequenceClassification.forward`,  you can safely ignore this message.
***** Running Evaluation *****
  Num examples = 787
  Batch size = 32


metrics:

accuracy: 0.6302414231257941,

f1: 0.6218474879106397,

precision: 0.6250108596871455,

recall: 0.6302414231257941,



## 5

Тут три рандомных отзыва на Крестного отца. Если посмотрим чуть выше, туда, где переводили оценки в категории, заметим, что 0 -- плохой отзыв, 1 -- нейтральный, 2 -- хороший. BERT это все перевел в лейблы, но так и остально, что LABEL_0 -- плохой, LABEL_1 -- нейтральный, LABEL_2 -- хороший. Подаем последовательно плохой, нейтральный, хороший, ожидаем получить выход 0, 1, 2.

In [66]:
good_review = '''The best This film is the best in the world.
Not only forr a good story but also for smart words said by Vito Carleone and his son Michel whichs usnabout gangstar, political, and usual life.
No one can make such a move about gangstar life.'''
bad_review = '''ONLY HALF THE FILM COULD BE WATCHED The film stopped playing at 1:30, that was half way. It just freezed there. waste of money.'''
neutral_review = '''Well produced but boring. Still better than scorcese or sopranos rubbish though.'''

In [67]:
from transformers import TextClassificationPipeline
model_3.to('cpu')
pipe = TextClassificationPipeline(model=model_3, tokenizer=tokenizer)
pred = pipe([bad_review, neutral_review, good_review])
pred

[{'label': 'LABEL_0', 'score': 0.720552921295166},
 {'label': 'LABEL_1', 'score': 0.5583726167678833},
 {'label': 'LABEL_2', 'score': 0.7636069059371948}]

Ура, так и получилось! Я осознанно взяла довольно однозначные отзывы, в плохом много негативно окрашенных слов, в хорошем -- позитивно окрашенных, а в нейтральном есть и такие, и такие (кстати, его модель ожидаемо распознала с наименьшей уверенностью). Модель работает :)

## 4 (бонус)

То же самое, что в прошлых заданиях, но попросим модель выдать hidden_states для каждого слоя, и с каждого из них (для нас -- трех) возьмем эмбеддинг CLS токена; затем найдем среднее этих трех эмбеддингов для каждого случая, его соконкатенируем с выдачей pooled_output, а дальше обучим модель, как обычно.

In [166]:
class SentimentClassifier_4(nn.Module):

  def __init__(self, n_classes):
    super().__init__()
    config = BertConfig.from_pretrained(PRE_TRAINED_MODEL_NAME, output_hidden_states=True)
    self.bert = BertModel.from_pretrained(PRE_TRAINED_MODEL_NAME, config=config)
    self.drop = nn.Dropout(p=0.3)
    self.out = nn.Linear(self.bert.config.hidden_size*2, n_classes)
  
  def forward(self, input_ids, attention_mask, labels):
    last_hidden_state, pooled_output, hidden_states = self.bert(
      input_ids=input_ids,
      attention_mask=attention_mask,
      return_dict=False)
    hidden_cls_tokens = [(x[:,0,:])for x in hidden_states]
    hidden_cls_tokens = torch.stack((hidden_cls_tokens))
    cls_mean = torch.mean(hidden_cls_tokens, 0)
    output = self.drop(torch.cat((cls_mean, pooled_output), 1))
    return (loss_fn(self.out(output), labels), self.out(output))

In [167]:
model_4 = SentimentClassifier_4(class_num)
model_4 = model_4.to(device)

loading configuration file https://huggingface.co/prajjwal1/bert-tiny/resolve/main/config.json from cache at /root/.cache/huggingface/transformers/3cf34679007e9fe5d0acd644dcc1f4b26bec5cbc9612364f6da7262aed4ef7a4.a5a11219cf90aae61ff30e1658ccf2cb4aa84d6b6e947336556f887c9828dc6d
Model config BertConfig {
  "attention_probs_dropout_prob": 0.1,
  "classifier_dropout": null,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 128,
  "initializer_range": 0.02,
  "intermediate_size": 512,
  "layer_norm_eps": 1e-12,
  "max_position_embeddings": 512,
  "model_type": "bert",
  "num_attention_heads": 2,
  "num_hidden_layers": 2,
  "output_hidden_states": true,
  "pad_token_id": 0,
  "position_embedding_type": "absolute",
  "transformers_version": "4.17.0",
  "type_vocab_size": 2,
  "use_cache": true,
  "vocab_size": 30522
}

loading weights file https://huggingface.co/prajjwal1/bert-tiny/resolve/main/pytorch_model.bin from cache at /root/.cache/huggingface/transformers/1ee037c9e

In [168]:
trainer_4 = Trainer(
    model=model_4,
    args=training_args,
    train_dataset=train_tok,
    eval_dataset=test_tok,
    tokenizer=tokenizer,
    data_collator=data_collator,
    compute_metrics=compute_metrics
    )

trainer_4.train()

The following columns in the training set  don't have a corresponding argument in `SentimentClassifier_4.forward` and have been ignored: score, at, repliedAt, sentiment, __index_level_0__, token_type_ids, content, sortOrder, userName, appId, text, userImage, replyContent, thumbsUpCount, reviewCreatedVersion. If score, at, repliedAt, sentiment, __index_level_0__, token_type_ids, content, sortOrder, userName, appId, text, userImage, replyContent, thumbsUpCount, reviewCreatedVersion are not expected by `SentimentClassifier_4.forward`,  you can safely ignore this message.
/usr/local/lib/python3.7/dist-packages/transformers/optimization.py:309: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  FutureWarning,
***** Running training *****
  Num examples = 14171
  Num Epochs = 8
  Instantaneous batch size per device = 32
  Tot

Epoch,Training Loss,Validation Loss,Accuracy,F1,Precision,Recall
1,1.074000,0.955854,0.554569,0.521541,0.527339,0.554569
2,0.799900,0.840267,0.597716,0.570113,0.571148,0.597716
3,0.909400,0.803354,0.620558,0.602604,0.601252,0.620558
4,0.695500,0.787622,0.629442,0.616324,0.613532,0.629442
5,0.969900,0.776068,0.635787,0.628817,0.625686,0.635787
6,0.668900,0.768535,0.637056,0.632570,0.630268,0.637056
7,0.768600,0.764942,0.643401,0.637006,0.634141,0.643401
8,0.618900,0.763779,0.642132,0.636237,0.633508,0.642132


The following columns in the evaluation set  don't have a corresponding argument in `SentimentClassifier_4.forward` and have been ignored: score, at, repliedAt, sentiment, __index_level_0__, token_type_ids, content, sortOrder, userName, appId, text, userImage, replyContent, thumbsUpCount, reviewCreatedVersion. If score, at, repliedAt, sentiment, __index_level_0__, token_type_ids, content, sortOrder, userName, appId, text, userImage, replyContent, thumbsUpCount, reviewCreatedVersion are not expected by `SentimentClassifier_4.forward`,  you can safely ignore this message.
***** Running Evaluation *****
  Num examples = 788
  Batch size = 32
Saving model checkpoint to ./results/checkpoint-500
Trainer.model is not a `PreTrainedModel`, only saving its state dict.
tokenizer config file saved in ./results/checkpoint-500/tokenizer_config.json
Special tokens file saved in ./results/checkpoint-500/special_tokens_map.json
The following columns in the evaluation set  don't have a corresponding arg

TrainOutput(global_step=3544, training_loss=0.8493161983872106, metrics={'train_runtime': 192.5901, 'train_samples_per_second': 588.649, 'train_steps_per_second': 18.402, 'total_flos': 0.0, 'train_loss': 0.8493161983872106, 'epoch': 8.0})

In [153]:
metrics = trainer_4.evaluate(eval_dataset=val_tok, metric_key_prefix="test")
print(f'''metrics:\n
accuracy: {metrics['test_accuracy']},\n
f1: {metrics['test_f1']},\n
precision: {metrics['test_precision']},\n
recall: {metrics['test_recall']},\n''')

The following columns in the evaluation set  don't have a corresponding argument in `SentimentClassifier_4.forward` and have been ignored: score, at, repliedAt, sentiment, __index_level_0__, token_type_ids, content, sortOrder, userName, appId, text, userImage, replyContent, thumbsUpCount, reviewCreatedVersion. If score, at, repliedAt, sentiment, __index_level_0__, token_type_ids, content, sortOrder, userName, appId, text, userImage, replyContent, thumbsUpCount, reviewCreatedVersion are not expected by `SentimentClassifier_4.forward`,  you can safely ignore this message.
***** Running Evaluation *****
  Num examples = 787
  Batch size = 32


metrics:

accuracy: 0.6569250317662008,

f1: 0.6524994829624226,

precision: 0.6540563574476743,

recall: 0.6569250317662008,

